## Homework 4
### Cohort 1 Group7
#### Members: Simon Geller, Alex Kanstantsinau, Weixia Cheng, Mengxiao Li, Darshan Parvadiya

In [2]:
import numpy as np
import pandas as pd
from IPython.display import Markdown as md

In [3]:
df = pd.read_excel("Homework_4_Data_Paths.xlsx", skiprows=4, sheet_name="Sheet1")
df.rename(columns={df.columns[0]: '0', df.columns[1]: '1'}, inplace=True)
df = df.transpose()
df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,...,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000
1,0.034538,0.044079,0.023008,0.032062,0.028467,0.024166,0.029572,0.035916,0.022969,0.031206,...,0.035436,0.032518,0.035770,0.035488,0.031158,0.034946,0.033237,0.035058,0.026699,0.035858
2,0.036521,0.040815,0.016484,0.031301,0.040712,0.038415,0.041278,0.028297,0.032999,0.020541,...,0.026117,0.037780,0.040124,0.022153,0.024437,0.042816,0.041074,0.040484,0.034786,0.032768
3,0.032033,0.034202,0.026603,0.027659,0.038105,0.030390,0.037565,0.041517,0.037384,0.026227,...,0.028123,0.033549,0.038457,0.016132,0.029230,0.043301,0.045932,0.037863,0.029896,0.041746
4,0.032885,0.030071,0.045279,0.024445,0.037651,0.025908,0.036344,0.029626,0.046883,0.030435,...,0.023506,0.037130,0.035146,0.030222,0.035417,0.043595,0.047138,0.036093,0.048631,0.023957


1. Compute the average short-term interest rate over each path over the next ten
years (don’t forget to include the current value in the average) and use these values
to solve for the current prices of zero-coupon bonds with maturities ranging from
one to ten years (D(1), D(2), D(3), . . . , D(10)).


In [4]:
def get_discount(spot_ls):
    discount_ls = []
    for i in range(len(spot_ls)-1):
        avg_spot = spot_ls[:i+2].mean()
        d = np.exp(-avg_spot*(i+1))
        discount_ls.append(d)
    return discount_ls

discount_arr = np.zeros([10,200])
for index, column in df.items():
    discount_arr[:,index] = get_discount(column)

discount = np.mean(discount_arr,axis=1)
print(pd.Series(discount))

0    0.968848
1    0.936725
2    0.904764
3    0.873446
4    0.843000
5    0.813496
6    0.784974
7    0.757398
8    0.730767
9    0.705043
dtype: float64


2. Solve for the price of a five-year interest rate cap on the short-term rate with a
strike rate of .037. An interest rate cap consists of a series of five individual caplets,
one for each of the five annual dates. The cash flow for a caplet with maturity T is
$max(0, r_T − K)$, where $r_T$ is the short-term rate at time T and K is the strike rate.


In [5]:
class Rate_Option:
    def __init__(self, _maturity, _strike, _type):
        self.maturity = _maturity
        self.strike = _strike
        self.type = _type
        self.price = self.valuation()
    def valuation(self):
        pv_arr = np.zeros([200,self.maturity])
        for i in range(self.maturity):
            row = df.iloc[i+1]
            if self.type == "Call":
                cashflow = np.maximum(row-self.strike, 0)*100
            if self.type == "Put":
                cashflow = np.maximum(self.strike-row, 0)*100
            pv = cashflow * discount_arr[i,:]
            pv_arr[:,i] = pv
        price = np.mean(np.sum(pv_arr,axis=1))
        return price

In [6]:
Cap = Rate_Option(5, 0.037, "Call")
print(f"The calculated price for C(5, 0.037) is: {Cap.price:.5f}")

The calculated price for C(5, 0.037) is: 0.98612


3. Solve for the price of a five-year interest rate floor on the short-term rate with
a strike rate of .020. A floor consists of a series of five individual floorlets, one
for each of the five annual dates. The cash flow for a floorlet with maturity T is
max(0, K − rT ) using the same notation as in problem 2.


In [7]:
Floor = Rate_Option(5, 0.02, "Put")
print(f"The calculated price for P(5, 0.02) is: {Floor.price:.5f}")

The calculated price for P(5, 0.02) is: 0.02738


4. Solve for the price of a 10-year range accrual note. This is a note with a notional
of $100. Each year, the note pays a coupon equal to the short-term rate (set at the
end of the year) provided the short-term rate is between 2 and 4 percent, otherwise
it pays nothing.


In [8]:
l,h = 0.02,0.04
range_df = df.applymap(lambda x: x if l <= x <= h else 0)
pv = range_df.iloc[1:] * discount_arr
price = np.mean(np.sum(pv,axis=0))*100
print(f"The calculated price for AN(5, 0.02, 0.04) is: {price: .4f}")

The calculated price for AN(5, 0.02, 0.04) is:  18.4989


5. Solve for the price of a five-year inverse floater with a notional amount of 100
that pays two times the short-term rate (set at the beginning of each year) minus
6 percent. Thus, if the short-term rate is 4 percent at the beginning of a year, the
cash flow at the end of the year would be 2 × 4 - 6 = 2 per $100 notional.


In [9]:
maturity = 5
inverse_df = df.iloc[1:maturity+1].applymap(lambda x: 2*x-0.06)
pv = inverse_df * discount_arr[:maturity,:]
price = np.mean(np.sum(pv,axis=0))*100
print(f'The calculated price for $IF(5, 0.06)$ is: {price:.4f}')

The calculated price for $IF(5, 0.06)$ is: 4.4648


6. Which is more valuable? A five-year European call option (caplet) on the short-term interest rate or a five-year European call option on the average short-term
interest rate during the five years? Assume that the strike rate is 0.04.

In [11]:
class Call_Option:
    def __init__(self, _maturity, _strike, _type):
        self.maturity = _maturity
        self.strike = _strike
        self.type = _type
        self.price = self.valuation()
    def valuation(self):
        if self.type == "Avg":
            rate = np.mean(df.iloc[:self.maturity+1],axis=0)
        if self.type == "Reg":
            rate = df.iloc[self.maturity]
        cashflow = np.maximum(rate-self.strike, 0)*100
        pv = cashflow * discount_arr[self.maturity-1,:]
        return pv.mean()

In [12]:
Avg_call = Call_Option(5,0.04,'Avg')
Reg_call = Call_Option(5,0.04,'Reg')
print(f'The calculated price for $AvgCall(5, 0.04)$ is: {Avg_call.price:.4f}')
print(f'The calculated price for $RegCall(5, 0.04)$ is: {Reg_call.price:.4f}')

The calculated price for $AvgCall(5, 0.04)$ is: 0.0135
The calculated price for $RegCall(5, 0.04)$ is: 0.1392
